In [1]:
import geemap
import ee

In [2]:
# Authenticate to Earth Engine
ee.Authenticate()

# Initialize the Earth Engine module.
ee.Initialize()

In [66]:
# Define the region of interest
roi = ee.Geometry.Rectangle(76.9844, 9.5178, 77.2894, 9.996260)

# Define the sentinal image collection and filter by date
dataset = ee.ImageCollection("COPERNICUS/S3/OLCI")\
     .select(['Oa08_radiance'])\
     .filterBounds(roi)\
    .filterDate('2022-01-01', '2022-01-5')
#    
# Convert to radiance units

# Function to multiply by conversion factor
def multiply_by_conversion_factor(image):
    conversion_factors = ee.Image([0.00876539])
    return image.multiply(conversion_factors)

# Apply the multiplication function to all images in the filtered collection
radiance_collection = dataset.map(multiply_by_conversion_factor)




### Download images

In [67]:
image_ids=radiance_collection.aggregate_array('system:index').getInfo()
print(len(image_ids))
# Export each image in the filtered collection
for i , image_id in enumerate(image_ids):
    image=ee.Image(radiance_collection.filter(ee.Filter.eq('system:index',image_id)).first())
    
    task = ee.batch.Export.image.toDrive(
    image=image,
    description=f'image_download_{i}',
    folder='sentinal_data',
    scale=300,
    region=roi ,)
    task.start()
    print(f'started task {i+1}')

9
started task 1
started task 2
started task 3
started task 4
started task 5
started task 6
started task 7
started task 8
started task 9
